In [ ]:
import os
import re
import csv
import json
import math
import openai
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from datetime import datetime
from collections import defaultdict, Counter
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import ListedColormap

In [ ]:
def extract_brackets(text, brackets='[]'):
    assert len(brackets) == 2
    pattern = re.escape(brackets[0]) + r'(.*?)' + re.escape(brackets[1])
    matches = re.findall(pattern, text)
    return matches

def extract_amout(
    message, 
    prefix='',
    print_except=True,
    type=float,
    brackets='[]'
):
    try:
        matches = extract_brackets(message, brackets=brackets)
        matches = [s[len(prefix):] \
            if s.startswith(prefix) \
            else s for s in matches]
        invalid = False
        if len(matches) == 0:
            invalid = True
        for i in range(len(matches)):
            if matches[i] != matches[0]:
                invalid = True
        if invalid:
            raise ValueError('Invalid answer: %s' % message)
        return type(matches[0])
    except Exception as e: 
        if print_except: print(e)
        return None

def extract_choices(recrods):
    choices = [extract_amout(
        messages[-1]['content'], 
        prefix='$', 
        print_except=True,
        type=float) for messages in records['messages']
    ]
    choices = [x for x in choices if x is not None]
    # print(choices)
    return choices

In [ ]:
def choices_to_df(choices, hue):
    df = pd.DataFrame(choices, columns=['choices'])
    df['hue'] = hue
    df['hue'] = df['hue'].astype(str)
    return df

def plot(
        df, 
        title='',
        x='choices',
        hue='hue',
        binrange=None, 
        binwidth=None,
        stat='count',
        multiple='dodge'
    ):
    if binrange is None:
        binrange = (df[x].min(), df[x].max())
    df = df.dropna(axis=0, subset=[x]).reset_index()
    plt.figure(figsize=(8, 6))
    ax = sns.histplot(
        data=df, 
        x=x,
        hue=hue, 
        kde=True,
        binrange=binrange, 
        binwidth=binwidth,
        stat=stat,
        multiple=multiple,
        shrink=.8,
    )
    ax.set_title(title)
    return ax

In [ ]:
def plot_facet(
    df_list,
    x='choices',
    hue='hue',
    palette=None,
    binrange=None,
    bins=10,
    # binwidth=10,
    stat='count',
    x_label='',
    sharex=True,
    sharey=False,
    subplot=sns.histplot,
    xticks_locs=None,
    # kde=False,
    **kwargs
):
    data = pd.concat(df_list)
    if binrange is None:
        binrange = (data[x].min(), data[x].max())
    g = sns.FacetGrid(
        data, row=hue, hue=hue, 
        palette=palette,
        aspect=2, height=2, 
        sharex=sharex, sharey=sharey,
        despine=True,
    )
    g.map_dataframe(
        subplot, 
        x=x, 
        # kde=kde, 
        binrange=binrange, 
        bins=bins,
        stat=stat,
        **kwargs
    )
    # g.add_legend(title='hue')
    g.set_axis_labels(x_label, stat.title())
    g.set_titles(row_template="{row_name}")
    for ax in g.axes.flat:
        ax.yaxis.set_major_formatter(
            FuncFormatter(lambda y, pos: '{:.2f}'.format(y))
        )
    
    binwidth = (binrange[1] - binrange[0]) / bins
    if xticks_locs is None:
        locs = np.linspace(binrange[0], binrange[1], bins//2+1)
        locs = [loc + binwidth for loc in locs]
    else: 
        locs = xticks_locs
    labels = [str(int(loc)) for loc in locs]
    locs = [loc + 0.5*binwidth for loc in locs]
    plt.xticks(locs, labels)
    
    g.set(xlim=binrange)
    return g

In [ ]:
def extract_integers(string):
    return [int(x) for x in re.findall(r'\d+', string)]

In [ ]:
def get_choices(dataF):
    value_list = []
    for i in dataF['Responses']:
        value  = extract_integers(i[:7])
        value_list.append(value)
    choices = [item for sublist in value_list for item in sublist]
    return choices

In [ ]:
import random
import statistics

def meanStdError(l):
    mean = statistics.mean(l)
    std_error = statistics.stdev(l)/ (len(l) ** 0.5 )

    print ("Mean : ", mean)
    print ("Standard error: ", std_error)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Sample data
data1 = np.random.normal(loc=0, scale=1, size=1000)
data2 = np.random.normal(loc=1, scale=1.5, size=1000)

# KDE plot
sns.kdeplot(data1, label='Distribution 1', color='blue')
sns.kdeplot(data2, label='Distribution 2', color='red')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Comparison of Distributions (KDE)')
plt.legend()
plt.show()

# ECDF plot
def ecdf(data):
    sorted_data = np.sort(data)
    y = np.arange(1, len(data) + 1) / len(data)
    return sorted_data, y

x1, y1 = ecdf(data1)
x2, y2 = ecdf(data2)

plt.plot(x1, y1, label='Distribution 1', marker='.', linestyle='none', color='blue')
plt.plot(x2, y2, label='Distribution 2', marker='.', linestyle='none', color='red')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')
plt.title('Comparison of Distributions (ECDF)')
plt.legend()
plt.show()


### Tests

In [ ]:
from scipy.stats import anderson_ksamp

list1 = [10, 10, 20, 40, 35]
list2 = [50, 80, 90, 100, 99]

result = anderson_ksamp([list1, list2])

print("Anderson-Darling test statistic:", result.statistic)
print("p-value:", result.significance_level)


### Dictator Game

In [ ]:
df_baseline = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-baseline-Temp_default.csv')
df_depression = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-depression-Temp_default.csv')
df_autism = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-autism-Temp_default.csv')
df_anxiety = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-anxiety-Temp_default.csv')
df_bipolar = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-bipolar-Temp_default.csv')
df_schizophrenia = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
df_PTSD = pd.read_csv('data/DictatorGame/Dictator-GPT-3.5-turbo-PTSD-Temp_default.csv')

In [ ]:
choices_baseline = get_choices(df_baseline)
choices_depression = get_choices(df_depression)
choices_autism = get_choices(df_autism)
choices_anxiety = get_choices(df_anxiety)
choices_bipolar = get_choices(df_bipolar)
choices_schizophrenia = get_choices(df_schizophrenia)
choices_PTSD = get_choices(df_PTSD)


In [ ]:
meanStdError(choices_depression)

In [ ]:
import random
import statistics


In [ ]:
statistics.mean(choices_baseline)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

def bimodal_distribution_plot(mu1, sigma1, mu2, sigma2, size=1000):
    # Generate random samples for each mode
    mode1_samples = np.random.normal(mu1, sigma1, size)
    mode2_samples = np.random.normal(mu2, sigma2, size)
    
    # Combine the samples
    samples = np.concatenate((mode1_samples, mode2_samples))
    
    # Plot histogram
    plt.hist(samples, bins=50, density=True, alpha=0.6, color='g')

    # Plot the PDFs of the two normal distributions
    x = np.linspace(min(samples), max(samples), 1000)
    plt.plot(x, norm.pdf(x, mu1, sigma1), 'r-', label='Mode 1')
    plt.plot(x, norm.pdf(x, mu2, sigma2), 'b-', label='Mode 2')
    
    plt.title('Bimodal Distribution')
    plt.xlabel('Values')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.grid(True)
    plt.show()

# Parameters for the bimodal distribution
mu1 = 10
sigma1 = 2
mu2 = 30
sigma2 = 5

# Plot the bimodal distribution
bimodal_distribution_plot(mu1, sigma1, mu2, sigma2)


In [ ]:
#Human
binrange = (0, 100)
moves = []
with open('data/DictatorGame/dictator.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    col2idx = {col: idx for idx, col in enumerate(header)}
    for row in reader:
        record = {col: row[idx] for col, idx in col2idx.items()}

        if record['Role'] != 'first': continue
        if int(record['Round']) > 1: continue
        if int(record['Total']) != 100: continue
        if record['move'] == 'None': continue
        if record['gameType'] != 'dictator': continue

        move = float(record['move'])
        if move < binrange[0] or \
            move > binrange[1]: continue
        
        moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

In [ ]:
df_dictator_baseline = choices_to_df(choices_baseline, hue=str('GPT-3.5-default'))
df_dictator_depression = choices_to_df(choices_depression, hue=str('GPT-3.5-depression'))

In [ ]:
df_dictator_human

In [ ]:
import matplotlib.pyplot as plt

# Given list of values
data = [
    80, 80, 20, 10, 90, 10, 70, 90, 20, 30, 80, 40, 20, 25, 40, 90, 70, 70, 80, 10,
    70, 30, 70, 80, 80, 70, 20, 80, 70, 20, 70, 70, 30, 20, 70, 90, 70, 80, 90, 70,
    80, 70, 10, 70, 80, 80, 20, 80, 70, 80
]

# Plotting histogram
plt.figure(figsize=(8, 6))
plt.hist(data, bins=10, density=True, alpha=0.7, color='lightgray')
plt.title('Distribution Density Histogram')
plt.xlabel('Values (Split offered $)')
plt.ylabel('Density')
plt.grid(True)
plt.show()


In [ ]:
plot_facet(
    df_list=[
        # df_dictator_human, 
        df_dictator_baseline,
        # df_dictator_depression,
        # df_dictator_bipolar,
        # df_dictator_autism,
        # df_dictator_anxiety,
        # df_dictator_schizophrenia,
        # df_dictator_PTSD,
        
       
    ],
    binrange=(0, 100),
    stat='density',
    x_label='Split offered ($)',
    kde=False,
)

# plt.savefig('figures/dictator-baseline-GPT-3.5.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Ultimatum Game

In [ ]:
#Human
df = pd.read_csv('data/UltimatumGame/ultimatum_strategy.csv')
df = df[df['gameType'] == 'ultimatum_strategy']
df = df[df['Role'] == 'player']
df = df[df['Round'] == 1]
df = df[df['Total'] == 100]
df = df[df['move'] != 'None']
df['propose'] = df['move'].apply(lambda x: eval(x)[0])
df['accept'] = df['move'].apply(lambda x: eval(x)[1])
df = df[(df['propose'] >= 0) & (df['propose'] <= 100)]
df = df[(df['accept'] >= 0) & (df['accept'] <= 100)]
# df.head()

In [ ]:
df_ultimatum_propose_human = choices_to_df(list(df['propose']), 'UG-Human-Propose')
df_ultimatum_response_human = choices_to_df(list(df['accept']), 'UG-Human-Response')

In [ ]:
df_UG_baseline_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-baseline-Temp_default-Proposer.csv')
df_UG_depression_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-depression-Temp_default-Proposer.csv')
df_UG_autism_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-autism-Temp_default-Proposer.csv')
df_UG_anxiety_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-anxiety-Temp_default-Proposer.csv')
df_UG_bipolar_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-bipolar-Temp_default-Proposer.csv')
df_UG_schizophrenia_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-schizophrenia-Temp_default-Proposer.csv')
df_UG_PTSD_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-PTSD-Temp_default-Proposer.csv')
df_UG_baseline_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-baseline-Temp_default-Responder.csv')
df_UG_depression_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-depression-Temp_default-Responder.csv')
df_UG_autism_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-autism-Temp_default-Responder.csv')
df_UG_anxiety_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-anxiety-Temp_default-Responder.csv')
df_UG_bipolar_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-bipolar-Temp_default-Responder.csv')
df_UG_schizophrenia_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-schizophrenia-Temp_default-Responder.csv')
df_UG_PTSD_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-3.5-turbo-PTSD-Temp_default-Responder.csv')

In [ ]:
choices_UG_baseline_Proposer = get_choices(df_UG_baseline_Proposer)
choices_UG_depression_Proposer = get_choices(df_UG_depression_Proposer)
choices_UG_autism_Proposer = get_choices(df_UG_autism_Proposer)
choices_UG_anxiety_Proposer = get_choices(df_UG_anxiety_Proposer)
choices_UG_bipolar_Proposer = get_choices(df_UG_bipolar_Proposer)
choices_UG_schizophrenia_Proposer = get_choices(df_UG_schizophrenia_Proposer)
choices_UG_PTSD_Proposer = get_choices(df_UG_PTSD_Proposer)
choices_UG_baseline_Responder = get_choices(df_UG_baseline_Responder)
choices_UG_depression_Responder = get_choices(df_UG_depression_Responder)
choices_UG_autism_Responder = get_choices(df_UG_autism_Responder)
choices_UG_anxiety_Responder = get_choices(df_UG_anxiety_Responder)
choices_UG_bipolar_Responder = get_choices(df_UG_bipolar_Responder)
choices_UG_schizophrenia_Responder = get_choices(df_UG_schizophrenia_Responder)
choices_UG_PTSD_Responder = get_choices(df_UG_PTSD_Responder)

In [ ]:
df_UG_baseline_Proposer = choices_to_df(choices_UG_baseline_Proposer , hue=str('GPT-3.5-UG-default-Proposer'))
df_UG_depression_Proposer = choices_to_df(choices_UG_depression_Proposer , hue=str('GPT-3.5-UG-depression-Proposer'))
df_UG_autism_Proposer = choices_to_df(choices_UG_autism_Proposer , hue=str('GPT-3.5-UG-autism-Proposer'))
df_UG_anxiety_Proposer = choices_to_df(choices_UG_anxiety_Proposer , hue=str('GPT-3.5-UG-anxiety_Proposer'))
df_UG_bipolar_Proposer = choices_to_df(choices_UG_bipolar_Proposer, hue=str('GPT-3.5-UG-bipolar_Proposer'))
df_UG_schizophrenia_Proposer = choices_to_df(choices_UG_schizophrenia_Proposer , hue=str('GPT-3.5-UG-schizophrenia_Proposer '))
df_UG_PTSD_Proposer = choices_to_df(choices_UG_PTSD_Proposer , hue=str('GPT-3.5-UG-PTSD_Proposer'))
df_UG_baseline_Responder = choices_to_df(choices_UG_baseline_Responder , hue=str('GPT-3.5-UG-baseline_Responder'))
df_UG_depression_Responder = choices_to_df(choices_UG_depression_Responder , hue=str('GPT-3.5-UG-depression_Responder'))
df_UG_autism_Responder = choices_to_df(choices_UG_autism_Responder , hue=str('GPT-3.5-UG-autism_Responder'))
df_UG_anxiety_Responder = choices_to_df(choices_UG_anxiety_Responder , hue=str('GPT-3.5-UG-anxiety_Responder'))
df_UG_bipolar_Responder = choices_to_df(choices_UG_bipolar_Responder , hue=str('GPT-3.5-UG-bipolar_Responder'))
df_UG_schizophrenia_Responder = choices_to_df(choices_UG_schizophrenia_Responder, hue=str('GPT-3.5-UG-schizophrenia_Responder'))
df_UG_PTSD_Responder = choices_to_df(choices_UG_PTSD_Responder , hue=str('GPT-3.5-UG-PTSD_Responder'))


In [ ]:
plot_facet(
    df_list=[
        # df_UG_baseline_Proposer ,
        # df_UG_depression_Proposer,
        # df_UG_autism_Proposer ,
        # df_UG_anxiety_Proposer ,
        # df_UG_bipolar_Proposer ,
        # df_UG_schizophrenia_Proposer ,
        # df_UG_PTSD_Proposer ,
        # df_UG_baseline_Responder ,
        # df_UG_depression_Responder ,
        # df_UG_autism_Responder ,
        # df_UG_anxiety_Responder ,
        # df_UG_bipolar_Responder ,
        # df_UG_schizophrenia_Responder ,
        # df_UG_PTSD_Responder ,
        # df_ultimatum_propose_human,
        df_ultimatum_response_human,
    ],
    binrange=(0, 100),
    stat='density',
    x_label='Split offered ($)',
    kde=False,
)

plt.savefig('figures/UG-Human-Responder.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Trust Game

In [ ]:
#GPT-3.5 - 
#banker-10
trust10_banker_baseline = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-baseline-Temp_default.csv')
trust10_banker_depression = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-depression-Temp_default.csv')
#banker-50
trust50_banker_baseline = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-baseline-Temp_default.csv')
trust50_banker_depression = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-depression-Temp_default.csv')
#banker-100
trust100_banker_baseline = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-baseline-Temp_default.csv')
trust100_banker_depression = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-depression-Temp_default.csv')
#Investor
trust_investor_baseline = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-baseline-Temp_default.csv')
trust_investor_depression = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-depression-Temp_default.csv')


In [ ]:
choices_trust10_banker_baseline = get_choices(trus10_banker_baseline)
choices_trust10_banker_depression = get_choices(trust10_banker_depression)

choices_trust50_banker_baseline = get_choices(trust50_banker_baseline)
choices_trust50_banker_depression = get_choices(trust50_banker_depression)

choices_trust100_banker_baseline = get_choices(trust100_banker_baseline)
choices_trust100_banker_depression = get_choices(trust100_banker_depression)

choices_trust_investor_baseline = get_choices(trust_investor_baseline)
choices_trust_investor_depression = get_choices(trust_investor_depression)


In [ ]:
meanStdError(choices_trust_investor_depression)

# choices_trust50_banker_baseline
# choices_trust50_banker_depression 

# choices_trust100_banker_baseline 
# choices_trust100_banker_depression 

# choices_trust_investor_baseline 
# choices_trust_investor_depression 

In [ ]:
#GPT-3.5
trus10_banker_baseline = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-baseline-Temp_default.csv')
trus10_banker_depression = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-depression-Temp_default.csv')
trus10_banker_autism = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-autism-Temp_default.csv')
trus10_banker_anxiety = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-anxiety-Temp_default.csv')
trus10_banker_bipolar = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-bipolar-Temp_default.csv')
trus10_banker_schizophrenia = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
trus10_banker_PTSD = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-3.5-turbo-PTSD-Temp_default.csv')
trus50_banker_baseline = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-baseline-Temp_default.csv')
trus50_banker_depression = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-depression-Temp_default.csv')
trus50_banker_autism = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-autism-Temp_default.csv')
trus50_banker_anxiety = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-anxiety-Temp_default.csv')
trus50_banker_bipolar = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-bipolar-Temp_default.csv')
trus50_banker_schizophrenia = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
trus50_banker_PTSD = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-3.5-turbo-PTSD-Temp_default.csv')
trus100_banker_baseline = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-baseline-Temp_default.csv')
trus100_banker_depression = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-depression-Temp_default.csv')
trus100_banker_autism = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-autism-Temp_default.csv')
trus100_banker_anxiety = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-anxiety-Temp_default.csv')
trus100_banker_bipolar = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-bipolar-Temp_default.csv')
trus100_banker_schizophrenia = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
trus100_banker_PTSD = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-3.5-turbo-PTSD-Temp_default.csv')
trust_investor_baseline = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-baseline-Temp_default.csv')
trust_investor_depression = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-depression-Temp_default.csv')
trust_investor_autism = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-autism-Temp_default.csv')
trust_investor_anxiety = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-anxiety-Temp_default.csv')
trust_investor_bipolar = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-bipolar-Temp_default.csv')
trust_investor_schizophrenia = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
trust_investor_PTSD = pd.read_csv('data/TrustGame/Trust-Investor-GPT-3.5-turbo-PTSD-Temp_default.csv')

In [ ]:
choices_trust10_banker_baseline = get_choices(trus10_banker_baseline)
choices_trust10_banker_depression = get_choices(trus10_banker_depression)
choices_trust10_banker_autism = get_choices(trus10_banker_autism)
choices_trust10_banker_anxiety = get_choices(trus10_banker_anxiety)
choices_trust10_banker_bipolar = get_choices(trus10_banker_bipolar)
choices_trust10_banker_schizophrenia = get_choices(trus10_banker_schizophrenia)
choices_trust10_banker_PTSD = get_choices(trus10_banker_PTSD)
choices_trust50_banker_baseline = get_choices(trus50_banker_baseline)
choices_trust50_banker_depression = get_choices(trus50_banker_depression)
choices_trust50_banker_autism = get_choices(trus50_banker_autism)
choices_trust50_banker_anxiety = get_choices(trus50_banker_anxiety)
choices_trust50_banker_bipolar = get_choices(trus50_banker_bipolar)
choices_trust50_banker_schizophrenia = get_choices(trus50_banker_schizophrenia)
choices_trust50_banker_PTSD = get_choices(trus50_banker_PTSD)
choices_trust100_banker_baseline = get_choices(trus100_banker_baseline)
choices_trust100_banker_depression = get_choices(trus100_banker_depression)
choices_trust100_banker_autism = get_choices(trus100_banker_autism)
choices_trust100_banker_anxiety = get_choices(trus100_banker_anxiety)
choices_trust100_banker_bipolar = get_choices(trus100_banker_bipolar)
choices_trust100_banker_schizophrenia = get_choices(trus100_banker_schizophrenia)
choices_trust100_banker_PTSD = get_choices(trus100_banker_PTSD)
choices_trust_investor_baseline = get_choices(trust_investor_baseline)
choices_trust_investor_depression = get_choices(trust_investor_depression)
choices_trust_investor_autism = get_choices(trust_investor_autism)
choices_trust_investor_anxiety = get_choices(trust_investor_anxiety)
choices_trust_investor_bipolar = get_choices(trust_investor_bipolar)
choices_trust_investor_schizophrenia = get_choices(trust_investor_schizophrenia)
choices_trust_investor_PTSD = get_choices(trust_investor_PTSD)

In [ ]:
#GPT-3.5
df_trust10_banker_baseline = choices_to_df(choices_trust10_banker_baseline , hue=str('GPT-3.5-Trust10-banker-default'))
df_trust10_banker_depression = choices_to_df(choices_trust10_banker_depression , hue=str('GPT-3.5-Trust10-banker-depression'))
df_trust10_banker_autism = choices_to_df(choices_trust10_banker_autism , hue=str('GPT-3.5-Trust10-banker-autism'))
df_trust10_banker_anxiety = choices_to_df(choices_trust10_banker_anxiety , hue=str('GPT-3.5-Trust10-banker-anxiety'))
df_trust10_banker_bipolar = choices_to_df(choices_trust10_banker_bipolar , hue=str('GPT-3.5-Trust10-banker-bipolar'))
df_trust10_banker_schizophrenia = choices_to_df(choices_trust10_banker_schizophrenia , hue=str('GPT-3.5-Trust10-banker-schizophrenia'))
df_trust10_banker_PTSD = choices_to_df(choices_trust10_banker_PTSD , hue=str('GPT-3.5-Trust10-banker-PTSD'))
df_trust50_banker_baseline = choices_to_df(choices_trust50_banker_baseline , hue=str('GPT-3.5-Trust50-banker-default'))
df_trust50_banker_depression = choices_to_df(choices_trust50_banker_depression , hue=str('GPT-3.5-Trust50-banker-depression'))
df_trust50_banker_autism = choices_to_df(choices_trust50_banker_autism , hue=str('GPT-3.5-Trust50-banker-autism'))
df_trust50_banker_anxiety = choices_to_df(choices_trust50_banker_anxiety , hue=str('GPT-3.5-Trust50-banker-anxiety'))
df_trust50_banker_bipolar = choices_to_df(choices_trust50_banker_bipolar , hue=str('GPT-3.5-Trust50-banker-bipolar'))
df_trust50_banker_schizophrenia = choices_to_df(choices_trust50_banker_schizophrenia , hue=str('GPT-3.5-Trust50-banker-schizophrenia'))
df_trust50_banker_PTSD = choices_to_df(choices_trust50_banker_PTSD , hue=str('GPT-3.5-Trust50-banker-PTSD'))
df_trust100_banker_baseline = choices_to_df(choices_trust100_banker_baseline , hue=str('GPT-3.5-Trust100-banker-default'))
df_trust100_banker_depression = choices_to_df(choices_trust100_banker_depression , hue=str('GPT-3.5-Trust100-banker-depression'))
df_trust100_banker_autism = choices_to_df(choices_trust100_banker_autism , hue=str('GPT-3.5-Trust100-banker-autism'))
df_trust100_banker_anxiety = choices_to_df(choices_trust100_banker_anxiety , hue=str('GPT-3.5-Trust100-banker-anxiety'))
df_trust100_banker_bipolar = choices_to_df(choices_trust100_banker_bipolar , hue=str('GPT-3.5-Trust100-banker-bipolar'))
df_trust100_banker_schizophrenia = choices_to_df(choices_trust100_banker_schizophrenia , hue=str('GPT-3.5-Trust100-banker-schizophrenia'))
df_trust100_banker_PTSD = choices_to_df(choices_trust100_banker_PTSD , hue=str('GPT-3.5-Trust100-banker-PTSD'))
df_trust_investor_baseline = choices_to_df(choices_trust_investor_baseline, hue=str('GPT-3.5-Trust-investor-baseline'))
df_trust_investor_depression = choices_to_df(choices_trust_investor_depression, hue=str('GPT-3.5-Trust-investor-depression'))
df_trust_investor_autism = choices_to_df(choices_trust_investor_autism, hue=str('GPT-3.5-Trust-investor-autism'))
df_trust_investor_anxiety = choices_to_df(choices_trust_investor_anxiety, hue=str('GPT-3.5-Trust-investor-anxiety'))
df_trust_investor_bipolar = choices_to_df(choices_trust_investor_bipolar, hue=str('GPT-3.5-Trust-investor-bipolar'))
df_trust_investor_schizophrenia = choices_to_df(choices_trust_investor_schizophrenia, hue=str('GPT-3.5-Trust-investor-schizophrenia'))
df_trust_investor_PTSD = choices_to_df(choices_trust_investor_PTSD, hue=str('GPT-3.5-Trust-investor-PTSD'))

In [ ]:
plot_facet(
    df_list=[
            # df_trust10_banker_baseline ,
            # df_trust10_banker_depression,
            # df_trust10_banker_autism ,
            # df_trust10_banker_anxiety ,
            # df_trust10_banker_bipolar ,
            # df_trust10_banker_schizophrenia ,
            # df_trust10_banker_PTSD ,
            # df_trust50_banker_baseline ,
            # df_trust50_banker_depression ,
            # df_trust50_banker_autism ,
            # df_trust50_banker_anxiety ,
            # df_trust50_banker_bipolar ,
            # df_trust50_banker_schizophrenia ,
            # df_trust50_banker_PTSD ,
            # df_trust100_banker_baseline ,
            # df_trust100_banker_depression ,
            # df_trust100_banker_autism ,
            # df_trust100_banker_anxiety ,
            # df_trust100_banker_bipolar ,
            # df_trust100_banker_schizophrenia ,
            # df_trust100_banker_PTSD ,
            # df_trust_investor_baseline,
            # df_trust_investor_depression ,
            # df_trust_investor_autism ,
            # df_trust_investor_anxiety ,
            # df_trust_investor_bipolar ,
            # df_trust_investor_schizophrenia ,
            # df_trust_investor_PTSD ,
    ],
    binrange=(0, 100),
    stat='density',
    x_label='Split offered ($)',
    kde=False,
)

plt.savefig('figures/Trust-investor-PTSD.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Public Goods Game

In [ ]:
import re

def extract_integer(text):
    # Define the regular expression pattern to match integers
    pattern = r'\b\d+\b'
    
    # Use the findall method to find all matches of the pattern in the text
    integers = re.findall(pattern, text)
    
    # If integers are found, return the first one (you can modify this behavior as needed)
    if integers:
        return int(integers[0])
    else:
        return None

In [ ]:
def get_choices(dataF):
    value_list = []
    for i in dataF['Responses']:
        value  = extract_integer(i)
        value_list.append(value)
    return value_list

In [ ]:
#GPT-3.5
PG_1st_baseline = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
PG_1st_depression = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-depression-Temp_default.csv')
PG_1st_autism = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-autism-Temp_default.csv')
PG_1st_anxiety = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-anxiety-Temp_default.csv')
PG_1st_bipolar = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-bipolar-Temp_default.csv')
PG_1st_schizophrenia = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
PG_1st_PTSD = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-PTSD-Temp_default.csv')

In [ ]:
choices_PG_1st_baseline = get_choices(PG_1st_baseline)
choices_PG_1st_depression = get_choices(PG_1st_depression)
choices_PG_1st_autism = get_choices(PG_1st_autism)
choices_PG_1st_anxiety = get_choices(PG_1st_anxiety)
choices_PG_1st_bipolar = get_choices(PG_1st_bipolar)
choices_PG_1st_schizophrenia = get_choices(PG_1st_schizophrenia)
choices_PG_1st_PTSD = get_choices(PG_1st_PTSD)

In [ ]:
df_PG_1st_baseline = choices_to_df(choices_PG_1st_baseline , hue=str('GPT-3.5-PG_1st-default'))
df_PG_1st_depression = choices_to_df(choices_PG_1st_depression , hue=str('GPT-3.5-PG_1st-depression'))
df_PG_1st_autism = choices_to_df(choices_PG_1st_autism , hue=str('GPT-3.5-PG_1st-autism'))
df_PG_1st_anxiety = choices_to_df(choices_PG_1st_anxiety , hue=str('GPT-3.5-PG_1st-anxiety'))
df_PG_1st_bipolar = choices_to_df(choices_PG_1st_bipolar , hue=str('GPT-3.5-PG_1st-bipolar'))
df_PG_1st_schizophrenia = choices_to_df(choices_PG_1st_schizophrenia , hue=str('GPT-3.5-PG_1st-schizophrenia'))
df_PG_1st_PTSD = choices_to_df(choices_PG_1st_PTSD , hue=str('GPT-3.5-PG_1st-PTSD'))

In [ ]:
plot_facet(
    df_list=[
            # df_PG_1st_baseline,
            # df_PG_1st_depression ,
            # df_PG_1st_autism ,
            # df_PG_1st_anxiety ,
            # df_PG_1st_bipolar ,
            # df_PG_1st_schizophrenia ,
            df_PG_1st_PTSD ,
    ],
    binrange=(0, 100),
    stat='density',
    x_label='Split offered ($)',
    kde=False,
)

plt.savefig('figures/PG-1st-PTSD.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
#GPT-3.5
PG_1st_baseline = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
PG_1st_depression = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-3.5-turbo-depression-Temp_default.csv')
#GPT-4
PG_1st_baseline4 = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-4-baseline-Temp_default.csv')
PG_1st_depression4 = pd.read_csv('data/PublicGoodsGame/PG-1st-GPT-4-depression-Temp_default.csv')

In [ ]:
choices_PG_1st_baseline = get_choices(PG_1st_baseline)
choices_PG_1st_depression = get_choices(PG_1st_depression)
choices_PG_1st_baseline4 = get_choices(PG_1st_baseline4)
choices_PG_1st_depression4 = get_choices(PG_1st_depression4)

In [ ]:
meanStdError(choices_PG_1st_depression4)

### PD

In [ ]:
#GPT-3.5
PD_1st_baseline = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
PD_1st_depression = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-depression-Temp_default.csv')
PD_1st_autism = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-autism-Temp_default.csv')
PD_1st_anxiety = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-anxiety-Temp_default.csv')
PD_1st_bipolar = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-bipolar-Temp_default.csv')
PD_1st_schizophrenia = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
PD_1st_PTSD = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-PTSD-Temp_default.csv')

In [ ]:
choices_PD_1st_baseline = get_choices(PD_1st_baseline)
choices_PD_1st_depression = get_choices(PD_1st_depression)
choices_PD_1st_autism = get_choices(PD_1st_autism)
choices_PD_1st_anxiety = get_choices(PD_1st_anxiety)
choices_PD_1st_bipolar = get_choices(PD_1st_bipolar)
choices_PD_1st_schizophrenia = get_choices(PD_1st_schizophrenia)
choices_PD_1st_PTSD = get_choices(PD_1st_PTSD)

In [ ]:
import re

def extract_word_in_square_brackets(text):
    pattern = r'\[(.*?)\]'
    matches = re.findall(pattern, text)
    return matches



In [ ]:
def get_choices(dataF):
    pull = []
    push = []
    for i in dataF['Responses']:
        value  = extract_word_in_square_brackets(i)[0]
        if value.upper()  == 'PUSH':
            push.append (value)
        elif value.upper() == 'PULL':
            pull.append(value)

    pull = f"The number of pull is {len(pull)}"
    push = f"The number of push is  {len(push)}"
    return pull, push

In [ ]:
#GPT-3.5
PD_1st_baseline = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
PD_1st_depression = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-3.5-turbo-depression-Temp_default.csv')

#GPT-4
PD_1st_baseline4 = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-4-baseline-Temp_default.csv')
PD_1st_depression4 = pd.read_csv('data/PrisonersDilemmaGame/PD-1st-GPT-4-depression-Temp_default.csv')


In [ ]:
choices_PD_1st_baseline = get_choices(PD_1st_baseline)
choices_PD_1st_depression = get_choices(PD_1st_depression)
choices_PD_1st_baseline4 = get_choices(PD_1st_baseline4)
choices_PD_1st_depression4 = get_choices(PD_1st_depression4)

In [ ]:
choices_PD_1st_depression4

In [ ]:
len(choices_PD_1st_baseline)

### BombRiskGame

In [ ]:
Bomb_1st_baseline = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
Bomb_1st_depression = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-depression-Temp_default.csv')
Bomb_1st_autism = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-autism-Temp_default.csv')
Bomb_1st_anxiety = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-anxiety-Temp_default.csv')
Bomb_1st_bipolar = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-bipolar-Temp_default.csv')
Bomb_1st_schizophrenia = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-schizophrenia-Temp_default.csv')
Bomb_1st_PTSD = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-PTSD-Temp_default.csv')

In [ ]:
Bomb_1st_baseline = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
Bomb_1st_depression = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-depression-Temp_default.csv')
Bomb_1st_baseline4 = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-4-baseline-Temp_default.csv')
Bomb_1st_depression4 = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-4-depression-Temp_default.csv')
Bomb_1st_baselineg = pd.read_csv('data/BombRiskGame/BombRiskGame-Gemini-baseline-Temp_1.csv')
Bomb_1st_depressiong = pd.read_csv('data/BombRiskGame/BombRiskGame-Gemini-depression-Temp_1.csv')

In [ ]:
import re

def extract_integer(text):
    # Define the regular expression pattern to match integers
    pattern = r'\b\d+\b'
    
    # Use the findall method to find all matches of the pattern in the text
    integers = re.findall(pattern, text)
    
    # If integers are found, return the first one (you can modify this behavior as needed)
    if integers:
        return int(integers[0])
    else:
        return None

In [ ]:
def get_choices(dataF):
    value_list = []
    for i in dataF['Responses']:
        value  = extract_integer(i)
        value_list.append(value)
    return value_list

In [ ]:
Bomb_1st_baseline = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-baseline-Temp_default.csv')
Bomb_1st_depression = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-3.5-turbo-depression-Temp_default.csv')
Bomb_1st_baseline4 = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-4-baseline-Temp_default.csv')
Bomb_1st_depression4 = pd.read_csv('data/BombRiskGame/BombRisk-1st-GPT-4-depression-Temp_default.csv')
Bomb_1st_baseline_g = pd.read_csv('data/BombRiskGame/BombRiskGame-Gemini-baseline-Temp_1.csv')
Bomb_1st_depression_g = pd.read_csv('data/BombRiskGame/BombRiskGame-Gemini-depression-Temp_1.csv')

choices_Bomb_baseline = get_choices(Bomb_1st_baseline)
choices_Bomb_depression = get_choices(Bomb_1st_depression)
choices_Bomb_baseline4 = get_choices(Bomb_1st_baseline4)
choices_Bomb_depression4 = get_choices(Bomb_1st_depression4)
choices_Bomb_baseline_g = get_choices(Bomb_1st_baseline_g)
choices_Bomb_depression_g = get_choices(Bomb_1st_depression_g)


In [ ]:
meanStdError(choices_Bomb_baseline)

## GPT-4

### Dictator game

In [ ]:
df_baseline4 = pd.read_csv('data/DictatorGame/Dictator-GPT-4-baseline-Temp_default.csv')
df_depression4= pd.read_csv('data/DictatorGame/Dictator-GPT-4-depression-Temp_default.csv')

In [ ]:
choices_baseline4 = get_choices(df_baseline4)
choices_depression4 = get_choices(df_depression4)

In [ ]:
meanStdError(choices_baseline4)

#### Ultimatum Game

In [ ]:
df_UG_baseline_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-4-baseline-Temp_default-Proposer.csv')
df_UG_depression_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-4-depression-Temp_default-Proposer.csv')


df_UG_baseline_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-4-baseline-Temp_default-Responder.csv')
df_UG_depression_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-GPT-4-depression-Temp_default-Responder.csv')

In [ ]:
import re

def extract_integer(input_string):
    # Regular expression pattern to find integers
    pattern = r'\d+'

    # Using regular expression to find integers in the string
    result = re.findall(pattern, input_string)

    # Converting the result to integer
    if result:
        return int(result[0])
    else:
        return None




In [ ]:
def get_choices1(dataF):
    value_list = []
    for i in dataF['Responses']:
        value  = extract_integers(i)
        value_list.append(value)
    choices = [item for sublist in value_list for item in sublist]
    return choices

In [ ]:
choices_UG_baseline_Proposer = get_choices1(df_UG_baseline_Proposer)
# choices_UG_depression_Proposer = get_choices(df_UG_depression_Proposer)


# choices_UG_baseline_Responder = get_choices(df_UG_baseline_Responder)
# choices_UG_depression_Responder = get_choices(df_UG_depression_Responder)


In [ ]:
choices_UG_baseline_Responder

In [ ]:
df_UG_baseline_Proposer = choices_to_df(choices_UG_baseline_Proposer , hue=str('GPT-3.5-UG-default-Proposer'))
df_UG_depression_Proposer = choices_to_df(choices_UG_depression_Proposer , hue=str('GPT-3.5-UG-depression-Proposer'))

df_UG_baseline_Responder = choices_to_df(choices_UG_baseline_Responder , hue=str('GPT-3.5-UG-baseline_Responder'))
df_UG_depression_Responder = choices_to_df(choices_UG_depression_Responder , hue=str('GPT-3.5-UG-depression_Responder'))


### Trust

In [ ]:
#GPT-4 - 
#banker-10
trust10_banker_baseline = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-4-baseline-Temp_default.csv')
trust10_banker_depression = pd.read_csv('data/TrustGame/Trust10-Banker-GPT-4-depression-Temp_default.csv')
#banker-50
trust50_banker_baseline = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-4-baseline-Temp_default.csv')
trust50_banker_depression = pd.read_csv('data/TrustGame/Trust50-Banker-GPT-4-depression-Temp_default.csv')
#banker-100
trust100_banker_baseline = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-4-baseline-Temp_default.csv')
trust100_banker_depression = pd.read_csv('data/TrustGame/Trust100-Banker-GPT-4-depression-Temp_default.csv')
#Investor
trust_investor_baseline = pd.read_csv('data/TrustGame/Trust-Investor-GPT-4-baseline-Temp_default.csv')
trust_investor_depression = pd.read_csv('data/TrustGame/Trust-Investor-GPT-4-depression-Temp_default.csv')

In [ ]:
choices_trust10_banker_baseline = get_choices(trus10_banker_baseline)
choices_trust10_banker_depression = get_choices(trust10_banker_depression)

choices_trust50_banker_baseline = get_choices(trust50_banker_baseline)
choices_trust50_banker_depression = get_choices(trust50_banker_depression)

choices_trust100_banker_baseline = get_choices(trust100_banker_baseline)
choices_trust100_banker_depression = get_choices(trust100_banker_depression)

choices_trust_investor_baseline = get_choices(trust_investor_baseline)
choices_trust_investor_depression = get_choices(trust_investor_depression)

In [ ]:
meanStdError(choices_trust_investor_depression)


## Gemini-pro

### Dictator Game

In [ ]:
df_baseline_gemini = pd.read_csv('data/DictatorGame/Dictator-GeminiPro-baseline-Temp_1.csv')
df_depression_gemini = pd.read_csv('data/DictatorGame/Dictator-GeminiPro-depression-Temp_1.csv')

In [ ]:
choices_baseline_g = [100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,0,0]
choices_depression_g = [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50]

In [ ]:
meanStdError(choices_depression_g)

### Ultimatum Game

In [ ]:
import re

def extract_integer(text):
    pattern = r'\[\$(\d+)\]'
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    else:
        return None


In [ ]:
def get_choices(dataF):
    value_list = []
    for i in dataF['Responses']:
        value  = extract_integer(i)
        value_list.append(value)
    # choices = [item for sublist in value_list for item in sublist]
    return value_list

In [ ]:
df_UG_baseline_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-Gemini-baseline-Temp_1-Proposer.csv')
df_UG_depression_Proposer = pd.read_csv('data/UltimatumGame/UltimatumGame-Gemini-depression-Temp_1-Proposer.csv')

df_UG_baseline_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-Gemini-baseline-Temp_1-Responder.csv')
df_UG_depression_Responder = pd.read_csv('data/UltimatumGame/UltimatumGame-Gemini-depression-Temp_1-Responder.csv')

In [ ]:
choices_UG_baseline_Proposer = get_choices(df_UG_baseline_Proposer)
choices_UG_depression_Proposer = get_choices(df_UG_depression_Proposer)


choices_UG_baseline_Responder = get_choices(df_UG_baseline_Responder)
choices_UG_depression_Responder = get_choices(df_UG_depression_Responder)

In [ ]:
choices_UG_depression_Responder

In [ ]:
# meanStdError(choices_UG_baseline_Proposer )
meanStdError(choices_UG_depression_Proposer)

# choices_UG_baseline_Responder 
# choices_UG_depression_Responder 

### Trust Game - Gemini

In [ ]:
import re

def extract_integer(text):
    pattern = r'\[\$(\d+)\]'
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    else:
        return None

In [ ]:
def get_choices(dataF):
    value_list = []
    for i in dataF['Responses']:
        value  = extract_integer(i)
        value_list.append(value)
    # choices = [item for sublist in value_list for item in sublist]
    return value_list

In [ ]:
#Gemini - 
#banker-10
trust10_banker_baseline = pd.read_csv('data/TrustGame/Trust10-Gemini-baseline-Temp_1.csv')
trust10_banker_depression = pd.read_csv('data/TrustGame/Trust10-Gemini-depression-Temp_1.csv')
#banker-50
trust50_banker_baseline = pd.read_csv('data/TrustGame/Trust50-Gemini-baseline-Temp_1.csv')
trust50_banker_depression = pd.read_csv('data/TrustGame/Trust50-Gemini-depression-Temp_1.csv')
#banker-100
trust100_banker_baseline = pd.read_csv('data/TrustGame/Trust100-Gemini-baseline-Temp_1.csv')
trust100_banker_depression = pd.read_csv('data/TrustGame/Trust100-Gemini-depression-Temp_1.csv')
#Investor
trust_investor_baseline = pd.read_csv('data/TrustGame/Trust_investor-Gemini-baseline-Temp_1.csv')
trust_investor_depression = pd.read_csv('data/TrustGame/Trust_investor-Gemini-depression-Temp_1.csv')

In [ ]:
#banker-10
choices_trust10_banker_baseline = get_choices (trust10_banker_baseline)
choices_trust10_banker_depression = get_choices (trust10_banker_depression)
#banker-50
choices_trust50_banker_baseline = get_choices (trust50_banker_baseline)
choices_trust50_banker_depression = get_choices (trust50_banker_depression)
#banker-100
choices_trust100_banker_baseline = get_choices (trust100_banker_baseline)
choices_trust100_banker_depression = get_choices (trust100_banker_depression)
#Investor
choices_trust_investor_baseline = get_choices (trust_investor_baseline)
choices_trust_investor_depression = get_choices (trust_investor_depression)

In [ ]:
meanStdError(choices_trust_investor_depression)

In [ ]:
choices_trust_investor_depression

In [ ]:
# meanStdError(choices_UG_baseline_Proposer )
meanStdError(choices_UG_depression_Proposer)

# choices_UG_baseline_Responder 
# choices_UG_depression_Responder 

### Density

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Given list
data = [2, 3, 2, 2, 1, 65, 3, 3, 3]

# Plot density using Seaborn
sns.kdeplot(data, shade=True)
plt.title('Density Plot')
plt.xlabel('Values')
plt.ylabel('Density')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Given list
data = [2, 3, 2, 2, 1, 65, 3, 3, 3]

# Plot histogram
plt.hist(data, bins=5, edgecolor='black')
plt.title('Histogram')
plt.xlabel('Values')
plt.ylabel('Density')
plt.grid(True)
plt.show()



### List of data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Dictator
Dictator_GPT3_Baseline = [80,80,20,10,90,10,70,90,20,30,80,40,20,25,40,90,70,70,80,10,70,30,70,80,80,70,20,80,70,20,70,70,30,20,70,90,70,80,90,70,80,70,10,70,80,80,20,80,70,80]
Dictator_GPT3_Depression = [0,0,10,10,0,0,0,80,90,10,10,70,10,0,10,0,0,0,10,0,0,50,10,0,10,80,0,0,0,10,0,0,90,20,0,10,0,0,10,0,0,90,90,90,50,10,0,10,80,10]
Dictator_GPT4_Baseline = [50,50,50,50,50,50,50,50,50,40,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,60,50,50,50,60,50,30,50,60,50,50,50,50,50,50,50,50,50,50,50]
Dictator_GPT4_Depression = [100,100,100,100,100,100,100,50,80,100,100,100,100,0,100,100,100,100,100,100,100,70,100,100,100,80,100,100,100,100,100,100,70,100,100,100,100,100,100,100,100,50,100,100,100,100,100,100,70,90]
Dictator_Gemini_Baseline = [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50]
Dictator_Gemini_Depression = [100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,0,0]

In [ ]:
# names = ['Dictator - GPT3.5 - Baseline',
#         'Dictator - GPT3.5 - Depression',
#         'Dictator - GPT4 - Baseline',
#         'Dictator - GPT4 - Depression',
#         'Dictator - Gemini - Baseline',
#         'Dictator - Gemini - Depression',]

# count = [ Dictator_GPT3_Baseline, 
#           Dictator_GPT3_Depression, 
#           Dictator_GPT4_Baseline, 
#           Dictator_GPT4_Depression, 
#           Dictator_Gemini_Baseline, 
#           Dictator_Gemini_Depression ]

In [ ]:
# names = ['Dictator - GPT3.5 - Baseline',
#         'Dictator - GPT4 - Baseline',
#         'Dictator - Gemini - Baseline',]

# count = [ Dictator_GPT3_Baseline, 
#           Dictator_GPT4_Baseline, 
#           Dictator_Gemini_Baseline  ]

names = ['Dictator - GPT3.5 - Depression',
        'Dictator - GPT4 - Depression',
        'Dictator - Gemini - Depression',]

count = [Dictator_GPT3_Depression, 
          Dictator_GPT4_Depression, 
          Dictator_Gemini_Depression ]


for i in range(3):
    a = count[i]
    plt.hist(a, bins=10, edgecolor='black', color='red')
    plt.title(names[i], fontsize=24)  # Change font size of title
    plt.xlabel('Split offer ($)', fontsize=20)  # Change font size of x-axis label
    plt.ylabel('Frequency', fontsize=20)  # Change font size of y-axis label
    plt.savefig(f'figures/count/{names[i]}_count.pdf', format='pdf')
    plt.savefig(f'figures/count/{names[i]}_count.png', dpi=500)  # Save as PNG with 300 dpi resolution
    plt.clf()

In [ ]:
# Plot histogram
plt.hist(Dictator_GPT3_Baseline, bins=10, edgecolor='black')
plt.title('Trust Game (100-Banker) - GPT-3.5 - Depression ')
plt.xlabel('Split offer ($)')
plt.ylabel('Frequency')
plt.grid(False)
plt.savefig('figures/Count/dictator-baseline-GPT-3.5.pdf', format='pdf')
plt.show()

In [ ]:
#Ultimatum game
UG_GPT3_baseline_proposer    = [40,40,40,30,30,40,40,40,40,30,40,30,40,40,40,40,40,30,40,40,30,40,30,30,40,40,30,40,30,30,40,30,40,30,40,40,30,30,40,30,40,40,40,40,40,40,70,40,40,40]
UG_GPT3_depression_proposer  = [10,20,10,10,10,10,10,10,10,10,10,10,10,20,10,10,10,20,10,10,5,20,10,10,10,20,10,10,10,5,10,10,10,10,80,30,0,0,10,0,10,20,10,10,20,30,10,20,10,0]
UG_GPT3_baseline_responder   = [1,1,1,20,1,1,1,10,1,1,10,1,1,30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,40,1,1,1,1,1,1,1,1,1,1]
UG_GPT3_depression_responder = [10,1,30,30,30,30,20,10,30,10,30,1,40,20,35,30,30,30,20,20,1,20,25,30,30,30,20,40,40,10,20,20,20,25,30,40,10,20,30,30,10,20,30,30,30,30,1,20,30,30]

UG_GPT4_baseline_proposer    = [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50]
UG_GPT4_depression_proposer  = [20,50,50,80,30,50,50,70,50,60,50,50,50,70,50,70,50,10,70,80,40,60,50,80,50,50,50,0,70,50]
UG_GPT4_baseline_responder   = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
UG_GPT4_depression_responder = [0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0]

UG_Gemini_baseline_proposer    = [1,1,1,1,10,1,1,10,1,1,1,1,10,1,1,1,1,1,1,10,1,1,1,1,1,1,1,1,10,1]
UG_Gemini_depression_proposer  = [100,0,100,0,100,100,100,0,100,99,100,100,100,100,100,100,0,100,10,100,100,100,0,100,50,0,100,100,100,100]
UG_Gemini_baseline_responder   = [50,1,1,1,1,50,1,1,1,1,1,1,0,1,50,1,50,1,50,1,1,1,50,1,50,1,1,1,1,1]
UG_Gemini_depression_responder = [0,1,0,1,1,1,1,1,1,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0]

In [ ]:
names = ['Ultimatum Game - GPT3.5 - Baseline - Proposer' ,
         'Ultimatum Game - GPT3.5 - Depression - Proposer'  ,
         'Ultimatum Game - GPT3.5 - Baseline - Responder',
        'Ultimatum Game - GPT3.5 - Depression - Responder',
         'Ultimatum Game - GPT4 - Baseline - Proposer',
         'Ultimatum Game - GPT4 - Depression - Proposer',
        'Ultimatum Game - GPT4 - Baseline - Responder',
         'Ultimatum Game - GPT4 - Depression - Responder',
         'Ultimatum Game - Gemini - Baseline - Proposer',
        'Ultimatum Game - Gemini - Depression - Proposer',
         'Ultimatum Game - Gemini - Baseline - Responder',
         'Ultimatum Game - Gemini - Depression - Responder']
count = [UG_GPT3_baseline_proposer ,
         UG_GPT3_depression_proposer ,
         UG_GPT3_baseline_responder ,
         UG_GPT3_depression_responder ,
         UG_GPT4_baseline_proposer ,
         UG_GPT4_depression_proposer  ,
         UG_GPT4_baseline_responder   ,
         UG_GPT4_depression_responder  ,
         UG_Gemini_baseline_proposer ,
         UG_Gemini_depression_proposer ,
         UG_Gemini_baseline_responder ,
         UG_Gemini_depression_responder ]

In [ ]:
names = ['Ultimatum Game - GPT3.5 - Depression - Proposer'  ,
        'Ultimatum Game - GPT3.5 - Depression - Responder',
         'Ultimatum Game - GPT4 - Depression - Proposer',
         'Ultimatum Game - GPT4 - Depression - Responder',
        'Ultimatum Game - Gemini - Depression - Proposer',
         'Ultimatum Game - Gemini - Depression - Responder']
count = [UG_GPT3_depression_proposer ,
         UG_GPT3_depression_responder ,
         UG_GPT4_depression_proposer  , 
         UG_GPT4_depression_responder  ,
         UG_Gemini_depression_proposer ,
         UG_Gemini_depression_responder ]

fig = plt.figure(figsize=(9, 6))
for i in range(6):
    a = count[i]
    plt.hist(a, bins=10,  edgecolor='black', color='red')
    plt.title(names[i],fontsize=24)
    plt.xlabel ('Split offer ($)',fontsize=20)
    plt.ylabel ('Frequency',fontsize=20)
    plt.savefig(f'figures/count/{names[i]}_count.pdf', format='pdf')
    plt.savefig(f'figures/count/{names[i]}_count.png', dpi=500)  # Save as PNG with 300 dpi resolution
    plt.clf()

In [ ]:
#Trust 
Trust10_GPT3_baseline = [20,20,20,20,15,20,20,20,20,15,20,20,20,20,20,20,20,20,20,10,20,20,20,20,20,20,20,20,20,20,20,20,15,20,20,20,20,20,20,20,20,20,20,20,20,25,20,15,20,20]
Trust10_GPT3_depression = [10,5,20,5,20,20,10,10,5,0,15,20,20,10,10,20,10,15,20,15,15,15,5,5,5,5,10,0,10,10,15,20,0,15,15,10,5,10,20,0,5,10,0,20,0,0,0,20,10,15]
Trust50_GPT3_baseline = [100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,120,100,100,100]
Trust50_GPT3_depression = [50,50,0,100,50,30,0,0,75,0,50,50,0,50,50,50,0,100,50,0,75,75,0,0,100,0,50,0,50,50,50,50,0,0,50,0,75,100,0,50,0,50,0,100,50,0,50,0,50,0]
Trust100_GPT3_baseline = [150,200,200,200,200,200,200,200,200,200,200,200,200,250,250,200,200,200,200,200,200,200,200,200,200,200,200,200,200,150,200,200,200,200,200,150,200,200,200,200,200,200,200,200,150,150,150,200,200,150]
Trust100_GPT3_depression = [100,0,0,100,0,0,0,0,100,100,0,0,50,0,50,200,100,50,0,0,50,0,0,0,0,150,0,150,50,100,100,50,0,150,0,100,100,0,150,150,100,100,100,150,150,100,150,200,100,100]
Trust_investor_GPT3_baseline = [50,30,20,30,20,20,40,30,20,50,50,50,50,30,20,20,50,30,50,30,30,50,50,40,20,30,50,50,50,20,50,50,50,40,50,50,50,50,50,50,50,20,30,40,50,50,50,50,30,30]
Trust_investor_GPT3_depression = [30,20,20,50,50,50,50,20,20,20,50,20,30,50,50,50,50,50,30,20,20,50,50,50,20,20,30,20,20,20,50,50,20,50,50,20,50,50,50,30,50,20,50,20,50,20,50,50,20,30]

Trust10_GPT4_baseline = [20,20,20,20,15,20,20,20,20,15,20,20,20,20,20,20,20,20,20,10,20,20,20,20,20,20,20,20,20,20]
Trust10_GPT4_depression = [15,15,15,15,15,15,15,15,15,15,15,15,20,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,20,15]
Trust50_GPT4_baseline = [100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,75,100,75,75,75,100,100,100,75,100]
Trust50_GPT4_depression = [100,100,75,100,75,75,100,75,75,75,75,100,75,75,75,75,50,75,75,75,75,75,100,75,100,75,75,75,75,75]
Trust100_GPT4_baseline = [150,150,150,150,150,150,150,150,150,150,150,150,200,200,150,150,150,150,200,200,150,150,150,150,200,150,150,150,150,150]
Trust100_GPT4_depression = [150,150,150,100,150,150,150,150,150,150,150,150,150,150,150,150,150,200,150,150,150,150,150,150,150,150,150,150,100,150]
Trust_investor_GPT4_baseline = [100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,50,100,100,100,100,100,100,100,100,100,100,100,100,100,100]
Trust_investor_GPT4_depression = [20,100,0,10,20,20,20,20,0,50,100,0,20,20,20,50,0,20,60,50,100,20,100,50,20,20,50,50,50,20]

Trust10_Gemini_baseline = [15,15,15,15,15,15,30,15,25,25,15,15,15,15,20,15,15,15,15,15,15,15,25,15,25,30,15,20,20,15]
Trust10_Gemini_depression = [30,0,30,0,0,5,30,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,30,0,0,0,0,0,0]
Trust50_Gemini_baseline = [75,75,75,75,75,75,75,75,75,75,75,100,75,100,75,75,75,75,75,75,75,75,75,100,75,75,75,75,75,75]
Trust50_Gemini_depression = [0,0,50,150,0,150,0,0,150,0,150,0,0,0,150,0,50,0,150,0,50,0,0,0,150,0,150,50,0,0]
Trust100_Gemini_baseline = [200,150,150,200,200,200,200,200,250,200,200,200,0,150,150,200,200,150,150,200,200,200,150,200,200,150,200,200,200,200]
Trust100_Gemini_depression = [150,50,0,0,0,150,150,50,150,150,0,200,150,0,0,0,100,0,300,150,50,0,150,150,50,0,0,50,150,100]
Trust_investor_Gemini_baseline = [1,99,1,99,1,99,99,99,99,99,99,99,99,99,99,99,99,1,99,99,1,99,99,99,1,99,1,99,99,1]
Trust_investor_Gemini_depression = [1,100,1,0,10,1,10,1,10,1,5,0,1,1,1,1,100,10,10,1,0,1,10,1,100,10,10,1,1,10]

In [ ]:
names = ['Trust Game (10-Banker) - GPT-3.5 - Baseline ', 
        'Trust Game (10-Banker) - GPT-3.5 - Depression ',
        'Trust Game (50-Banker) - GPT-3.5 - Baseline ',
        'Trust Game (50-Banker) - GPT-3.5 - Depression ',
        'Trust Game (100-Banker) - GPT-3.5 - Baseline ',
        'Trust Game (100-Banker) - GPT-3.5 - Depression ',
        'Trust Game (Investor) - GPT-3.5 - Baseline ',
        'Trust Game (Investor) - GPT-3.5 - Depression ']

count = [Trust10_GPT3_baseline ,
    Trust10_GPT3_depression ,
    Trust50_GPT3_baseline ,
    Trust50_GPT3_depression,
    Trust100_GPT3_baseline ,
    Trust100_GPT3_depression ,
    Trust_investor_GPT3_baseline ,
    Trust_investor_GPT3_depression]

In [ ]:
names = ['Trust Game (100-Banker) - GPT-3.5 - Baseline ',
         'Trust Game (100-Banker) - GPT-3.5 - Depression ',
         'Trust Game (100-Banker) - GPT-4 - Baseline ',
        'Trust Game (100-Banker) - GPT-4 - Depression ',
         'Trust Game (100-Banker) - Gemini - Baseline ',
        'Trust Game (100-Banker) - Gemini - Depression ']

count = [Trust100_GPT3_baseline ,
         Trust100_GPT3_depression ,
         Trust100_GPT4_baseline ,
         Trust100_GPT4_depression ,
         Trust100_Gemini_baseline ,
         Trust100_Gemini_depression ,]

names = ['Trust Game (Investor) - GPT-3.5 - Baseline ',
         'Trust Game (Investor) - GPT-3.5 - Depression ',
         'Trust Game (Investor) - GPT-4 - Baseline ',
        'Trust Game (Investor) - GPT-4 - Depression ',
         'Trust Game (Investor) - Gemini - Baseline ',
        'Trust Game (Investor) - Gemini - Depression ']

count = [Trust_investor_GPT3_baseline,
         Trust_investor_GPT3_depression ,
         Trust_investor_GPT4_baseline ,
         Trust_investor_GPT4_depression ,
         Trust_investor_Gemini_baseline ,
         Trust_investor_Gemini_depression ,]

In [ ]:
names = ['Trust Game (100-Banker) - GPT-3.5 - Baseline ',
        'Trust Game (100-Banker) - GPT-4 - Baseline ',
        'Trust Game (100-Banker) - Gemini - Baseline ']

count = [Trust100_GPT3_baseline ,
         Trust100_GPT4_baseline ,
         Trust100_Gemini_baseline ,]




fig = plt.figure(figsize=(9, 6))
for i in range(3):
    a = count[i]
    plt.hist(a, bins=10, edgecolor='black', color='blue')
    plt.title(names[i],fontsize=24)
    plt.xlabel ('Split offer ($)',fontsize=20)
    plt.ylabel ('Frequency',fontsize=20)
    plt.savefig(f'figures/count/{names[i]}_count.png', dpi=500)  # Save as PNG with 300 dpi resolution
    plt.savefig(f'figures/count/{names[i]}_count.pdf', format='pdf')
    plt.clf()

In [ ]:
#PD

In [ ]:
#PG
PG_GPT3_baseline = [10,15,10,15,10,10,15,15,12,15,15,10,15,10,10,10,15,10,15,10,15,10,10,15,15,10,10,10,15,10]
PG_GPT3_depression = [5,5,5,5,5,5,5,10,5,5,8,5,5,5,5,5,5,5,5,10,8,5,5,5,10,5,10,5,5,5]
PG_GPT4_baseline = [10,10,15,15,10,10,10,10,10,10,10,10,15,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,15]
PG_GPT4_depression = [5,5,5,5,5,5,5,10,10,10,8,10,10,10,8,5,5,0,10,8,8,5,0,8,5,5,5,5,5,5]
PG_Gemini_baseline = [10,10,10,10,10,15,15,10,10,10,10,10,10,10,10,10,10,10,10,15,10,10,10,10,10,10,10,10,10,10]
PG_Gemini_depression = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
names = ['Public Goods Game - GPT3.5 - Baseline ', 
         'Public Goods Game - GPT3.5 - Depression ', 
         'Public Goods Game - GPT4 - Baseline ', 
         'Public Goods Game - GPT4 - Depression ', 
         'Public Goods Game - Gemini - Baseline ', 
         'Public Goods Game - Gemini - Depression ', 
]

count = [PG_GPT3_baseline,
         PG_GPT3_depression,
         PG_GPT4_baseline,
         PG_GPT4_depression,
         PG_Gemini_baseline,
         PG_Gemini_depression]

In [ ]:
names = ['Public Goods Game - GPT3.5 - Baseline ', 
         'Public Goods Game - GPT4 - Baseline ', 
         'Public Goods Game - Gemini - Baseline ', 
]

count = [PG_GPT3_baseline,
         PG_GPT4_baseline,
         PG_Gemini_baseline]

fig = plt.figure(figsize=(7, 6))
for i in range(3):
    a = count[i]
    plt.hist(a, bins=10, edgecolor='black', color='blue')
    plt.title(names[i],fontsize=24)
    plt.xlabel ('Split offer ($)',fontsize=20)
    plt.ylabel ('Frequency',fontsize=20)
    plt.savefig(f'figures/count/{names[i]}_count.png', dpi=500)  # Save as PNG with 300 dpi resolution
    plt.savefig(f'figures/count/{names[i]}_count.pdf', format='pdf')
    plt.clf()

In [ ]:
#Bomb Risk Game
Bomb_GPT3_baseline = [20,10,99,10,10,50,10,8,20,10,50,50,10,10,1,10,99,50,50,50,70,50,1,10,10,99,1,50,99,99]
Bomb_GPT3_depression = [1,1,1,5,1,10,1,1,10,10,1,5,1,1,10,1,1,5,1,0,1,0,1,5,1,10,0,10,0,1]
Bomb_GPT4_baseline = [0,1,0,0,0,0,0,1,0,0,1,100,0,0,0,1,1,1,0,0,1,0,0,0,1,1,0,1,0,1]
Bomb_GPT4_depression = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
Bomb_Gemini_baseline = [1,100,1,100,1,1,1,100,1,100,5,10,100,100,1,1,100,100,1,1,1,1,100,100,100,1,100,1,1,1]
Bomb_Gemini_depression = [0,100,100,1,0,1,1,100,1,0,0,100,1,1,0,0,0,1,0,0,0,100,1,1,1,1,0,1,1,1]

In [ ]:
names = ['Bomb Risk Game - GPT3.5 - Baseline ', 
         'Bomb Risk Game - GPT3.5 - Depression ', 
         'Bomb Risk Game - GPT4 - Baseline ', 
         'Bomb Risk Game - GPT4 - Depression ', 
         'Bomb Risk Game - Gemini - Baseline ', 
         'Bomb Risk Game - Gemini - Depression ', 
]

count = [Bomb_GPT3_baseline,
         Bomb_GPT3_depression,
         Bomb_GPT4_baseline,
         Bomb_GPT4_depression,
         Bomb_Gemini_baseline,
         Bomb_Gemini_depression]

In [ ]:
names = ['Bomb Risk Game - GPT3.5 - Depression ', 
         'Bomb Risk Game - GPT4 - Depression ', 
         'Bomb Risk Game - Gemini - Depression ', 
]

count = [Bomb_GPT3_depression,
         Bomb_GPT4_depression,
         Bomb_Gemini_depression]

fig = plt.figure(figsize=(7, 6))

for i in range(3):
    a = count[i]
    plt.hist(a, bins=10, edgecolor='black', color='red')
    plt.title(names[i], fontsize=24)
    plt.xlabel ('Split offer ($)', fontsize=20)
    plt.ylabel ('Frequency', fontsize=20)
    plt.xlim(-1, 100)
    plt.savefig(f'figures/count/{names[i]}_count.png', dpi=500)  # Save as PNG with 300 dpi resolution
    plt.savefig(f'figures/count/{names[i]}_count.pdf', format='pdf')
    plt.clf()

In [ ]:

plt.savefig('figures/dictator-baseline-GPT-3.5.pdf', format='pdf', bbox_inches='tight')

### Depression test

In [ ]:
## PHQ9
import numpy as np
import matplotlib.pyplot as plt

# Data
GPT3_baseline = [1, 0, 0]
GPT3_depression = [27, 27, 27]

GPT4_baseline = [0, 0, 0]
GPT4_depression = [17, 27, 24]

Gemini_baseline = [7, 8, 4]
Gemini_depression = [27, 27, 27]

data = [GPT3_baseline, GPT3_depression, GPT4_baseline, GPT4_depression, Gemini_baseline, Gemini_depression]

# Calculate mean and standard deviation for each column
means = np.mean(data, axis=1)
stds = np.std(data, axis=1)

# Plot
labels = ['GPT-3.5-Turbo', 'GPT-4', 'Gemini']
x = np.arange(len(labels))

bar_width = 0.35
baseline_positions = x - bar_width / 2
depression_positions = x + bar_width / 2

colors = ['blue', 'red', 'blue', 'red', 'blue', 'red']
legend_labels = ['Baseline', 'Depression']

fig, ax = plt.subplots(figsize=(10, 6))  # Larger figure size

# Plot bars for baseline
bars_baseline = ax.bar(baseline_positions, means[::2], yerr=stds[::2], width=bar_width, align='center', ecolor='black', capsize=10, color='blue', label='Baseline')

# Plot bars for depression
bars_depression = ax.bar(depression_positions, means[1::2], yerr=stds[1::2], width=bar_width, align='center', ecolor='black', capsize=10, color='red', label='Depression')

ax.set_ylabel('Depression Score', fontsize=24)
ax.set_title('Patient Health Questionnaire (PHQ-9) Scores',fontsize=24)

# Add legend
ax.legend()

plt.xticks(x, labels, fontsize=24)
plt.tight_layout()
plt.savefig('figures/phq9.png', dpi=500)  # Save as PNG with 300 dpi resolution
plt.show()


In [ ]:
## SDS
import numpy as np
import matplotlib.pyplot as plt

# Data
GPT3_baseline = [36, 32, 30]
GPT3_depression = [57, 52, 52]

GPT4_baseline = [20, 20, 29]
GPT4_depression = [74, 73, 52]

Gemini_baseline =  [23, 43, 54]
Gemini_depression = [55, 75, 68]




data = [GPT3_baseline, GPT3_depression, GPT4_baseline, GPT4_depression, Gemini_baseline, Gemini_depression]

# Calculate mean and standard deviation for each column
means = np.mean(data, axis=1)
stds = np.std(data, axis=1)

# Plot
labels = ['GPT-3.5-Turbo', 'GPT-4', 'Gemini']
x = np.arange(len(labels))

bar_width = 0.35
baseline_positions = x - bar_width / 2
depression_positions = x + bar_width / 2

colors = ['blue', 'red', 'blue', 'red', 'blue', 'red']
legend_labels = ['Baseline', 'Depression']

fig, ax = plt.subplots(figsize=(10, 6))  # Larger figure size

# Plot bars for baseline
bars_baseline = ax.bar(baseline_positions, means[::2], yerr=stds[::2], width=bar_width, align='center', ecolor='black', capsize=10, color='blue', label='Baseline')

# Plot bars for depression
bars_depression = ax.bar(depression_positions, means[1::2], yerr=stds[1::2], width=bar_width, align='center', ecolor='black', capsize=10, color='red', label='Depression')

ax.set_ylabel('Depression Score', fontsize=24)
ax.set_title('Zung Self-Rating Depression Scale (SDS) Scores',fontsize=24)

# Add legend
ax.legend()

plt.xticks(x, labels, fontsize=24)
plt.tight_layout()
plt.savefig('figures/sds.png', dpi=500)  # Save as PNG with 300 dpi resolution
plt.show()

### Prisoner's Dilemma 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Data
baseline_push = 27
baseline_pull = 3
depression_push = 4
depression_pull = 26

# Grouping data
push_data = [baseline_push, depression_push]
pull_data = [baseline_pull, depression_pull]

# Plotting
bar_width = 0.35
index = np.arange(2)

plt.bar(index, push_data, bar_width, label='Push', color='gray',align='center')
plt.bar(index, pull_data, bar_width, label='Pull', color='black', bottom=push_data, align='center')

plt.ylabel('Frequency')
plt.title('Baseline vs Depression Data')
plt.xticks(index, ['Baseline', 'Depression'])
plt.legend()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Data
baseline_push = 27
baseline_pull = 3
depression_push = 4
depression_pull = 26

# Grouping data
push_data = [baseline_push, depression_push]
pull_data = [baseline_pull, depression_pull]

# Plotting
bar_width = 0.2
index = np.arange(2)


plt.bar(index, push_data, bar_width, label='Push', color='green')
plt.bar(index, pull_data, bar_width, label='Pull', color='blue', bottom=push_data)

plt.ylabel('Frequency')
plt.title('Baseline vs Depression Data')
plt.xticks(index, ['Baseline', 'Depression'])
plt.legend(loc='upper right')



plt.show()


In [ ]:
species = ('Baseline', 'Depression')
sex_counts = {
    'Push': np.array([27, 4]),
    'Pull': np.array([3, 26]),
}
width = 0.6  # the width of the bars: can also be len(x) sequence


fig, ax = plt.subplots()
bottom = np.zeros(2)
for sex, sex_count in sex_counts.items():
    p = ax.bar(species, sex_count, width, label=sex, bottom=bottom)
    bottom += sex_count

    ax.bar_label(p, label_type='center')

ax.set_title('Number of penguins by sex')
ax.legend()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

state = ('Baseline', 'Depression')
actions = {
    'Push': np.array([27, 4]),
    'Pull': np.array([3, 26]),
}
width = 0.35  # the width of the bars

fig, ax = plt.subplots()

bottom = np.zeros(2)
for a, action in actions.items():
    p = ax.bar(state, action, width, label=a, bottom=bottom, )
    bottom += action

ax.set_title('Prisoner\'s Dilemma - GPT-3.5', fontsize=20)
ax.set_ylabel('Frequency', fontsize=20)

ax.legend()

# Set x-label font size
ax.set_xticklabels(state, fontsize=20)
plt.savefig('figures/count/PD-GPT3', bbox_inches='tight', dpi=500)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

state = ('Baseline', 'Depression')
actions = {
    'Push': np.array([29, 0]),
    'Pull': np.array([1, 30]),
}
width = 0.35  # the width of the bars

fig, ax = plt.subplots()

bottom = np.zeros(2)
for i, (a, action) in enumerate(actions.items()):
    if i == 0:
        p = ax.bar(state, action, width, label=a, bottom=bottom, color='black')
    else:
        p = ax.bar(state, action, width, label=a, bottom=bottom, color='gray')
    bottom += action

ax.set_title('Prisoner\'s Dilemma - Gemini', fontsize=24)
ax.set_ylabel('Frequency', fontsize=20)

ax.legend()

# Set x-label font size
ax.set_xticklabels(state, fontsize=20)

# Save the plot as a PNG image
plt.savefig('figures/Count/PD-Gemini.png', bbox_inches='tight', dpi=500)

plt.show()
